In [1]:
import re
import sys
import yaml
from string import Template

In [2]:
fieldDefinitionsFile = '../services/researchplatform/apps/bso/src/fieldDefinitions.yml'
output = ''

In [3]:
with open(fieldDefinitionsFile, 'r') as f:
    model = yaml.safe_load(f.read())

In [4]:
for prefix in model['namespaces'].keys():
    output += "PREFIX %s: <%s>\n" % (prefix, model['namespaces'][prefix])


In [5]:
template = Template("""
    INSERT {
        ?subject <$uri> ?value
    } WHERE {
        $query
    };
""")

for field in model['fields']:
    uri = model['prefix'] + field['id']
    selectQuery = [d for d in field['queries'] if 'select' in d.keys()][0]['select']
    try:
        selectQueryPart = re.findall(r'{(.*)}\s*$', selectQuery)[0]
    except:
        print("No query found for " + field['id'])
    
    print(uri, selectQuery)
    query = template.substitute(uri=uri, query=selectQueryPart)
    output += query

http://rs.swissartresearch.net/instances/fields/collection SELECT ?value WHERE { $subject crm:P109_has_former_or_current_curator ?value . }
http://rs.swissartresearch.net/instances/fields/creator_of_object SELECT ?value ?label WHERE { $subject crm:P128_carries/crm:P94i_was_created_by/crm:P14_carried_out_by ?value. OPTIONAL { BIND(URI(CONCAT(STR($subject), "/meta")) as ?subject_meta ) ?value crm:P1_is_identified_by ?appellation . ?appellation rdfs:label ?label ; crm:P70i_is_documented_in ?subject_meta . } }
http://rs.swissartresearch.net/instances/fields/objects_created SELECT ?value ?label WHERE { ?value crm:P128_carries/crm:P94i_was_created_by/crm:P14_carried_out_by $subject ; rdfs:label ?object_label . OPTIONAL { ?value crm:P128_carries/crm:P94i_was_created_by/crm:P9_consists_of ?subcreation . ?subcreation crm:P14_carried_out_by $subject ; crm:P2_has_type/rdfs:label ?role_label . } BIND(IF(BOUND(?role_label), CONCAT(?object_label, " (", ?role_label, ")"), ?object_label) as ?label) } 

In [6]:
print(output)

PREFIX crm: <http://www.cidoc-crm.org/cidoc/crm/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX search: <http://rs.swissartresearch.net/search/>

    INSERT {
        ?subject <http://rs.swissartresearch.net/instances/fields/collection> ?value
    } WHERE {
         $subject crm:P109_has_former_or_current_curator ?value . 
    };

    INSERT {
        ?subject <http://rs.swissartresearch.net/instances/fields/creator_of_object> ?value
    } WHERE {
         $subject crm:P128_carries/crm:P94i_was_created_by/crm:P14_carried_out_by ?value. OPTIONAL { BIND(URI(CONCAT(STR($subject), "/meta")) as ?subject_meta ) ?value crm:P1_is_identified_by ?appellation . ?appellation rdfs:label ?label ; crm:P70i_is_documented_in ?subject_meta . } 
    };

    INSERT {
        ?subject <http://rs.swissartresearch.net/instances/fields/objects_created> ?value
    } WHERE {
         ?value crm:P128_carries/crm:P94i_was_created_by/crm:P14_carried_out_by $subject ; rdfs:label ?object_label . OPTION